<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA23_RNN%E6%8A%80%E5%B7%A7%E8%A8%8E%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot

### 1. 資料的輸入：文字怎麼輸入電腦

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/review.csv \
    -O /content/review.csv

--2022-04-10 17:01:54--  https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/review.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1141 (1.1K) [text/plain]
Saving to: ‘/content/review.csv’

/content/review.csv 100%[===================>]   1.11K  --.-KB/s    in 0s      

2022-04-10 17:01:54 (77.2 MB/s) - ‘/content/review.csv’ saved [1141/1141]



In [ ]:
df = pd.read_csv('review.csv')

In [ ]:
df.head()

,評論,正負評
0,蠻穩定的 行車記錄器就是要穩 其他其次,1
1,流媒體不錯 晚上又清楚,1
2,最近有裝 A129 pro duo，畫質不錯，之前mio 791d用兩年也沒遇到問題,1
3,mio 後鏡頭一直斷線 拉線要很注意 很麻煩,0
4,晚上只有cansonic z3這種有望遠鏡的才拍的到,1


找出中文的部分

In [ ]:
import re

In [ ]:
patn=re.compile(r"[\u4E00-\u9FD5]+")

In [ ]:
X = df['評論'][3]

In [ ]:
X

'mio 後鏡頭一直斷線 拉線要很注意 很麻煩'

In [ ]:
patn.findall(X)

['後鏡頭一直斷線', '拉線要很注意', '很麻煩']

In [ ]:
for review in df['評論']:
    print(patn.findall(review))

['蠻穩定的', '行車記錄器就是要穩', '其他其次']
['流媒體不錯', '晚上又清楚']
['最近有裝', '畫質不錯', '之前', '用兩年也沒遇到問題']
['後鏡頭一直斷線', '拉線要很注意', '很麻煩']
['晚上只有', '這種有望遠鏡的才拍的到']
['邁不錯']
['現在新的', '錄一錄會一直出現記憶卡損壞的語音就停止了', '要重裝記憶卡不知道是只有這台']
['邁後視鏡含夜光後視', '六月裝家裡兩台老車', '後視都裝在車外', '目前都正常', '便宜好選擇']
['我要跟你說', '他的導航是廢渣', '要用他的導航才能顯示左轉右轉和路名']
['以前用很久', '放', '還行']
['不過用久了', '像深灰色款的會慢慢變淡一點']
['我用了快五年了', '持續使用中']
['我覺得很好用']
['用過一款', '吋包超過十年', '很滿意']
['個人頗推這個牌子好看又耐用']
['上次買到', '折以後約', '年還在用']
['這個沒有', '充電呀']
['我剛好有買這款', '因為有三個', '插槽才買的', '質感還算不錯', '平常會外接螢幕使用', '會熱但不至於到燙', '跟充電都不會影響']


化資料為我們要的形式

In [ ]:
x_tmp = []

for review in df['評論']:
    sen_list = patn.findall(review)
    sen = ' '.join(sen_list)
    x_tmp.append(sen)

In [ ]:
x_tmp

['蠻穩定的 行車記錄器就是要穩 其他其次',
 '流媒體不錯 晚上又清楚',
 '最近有裝 畫質不錯 之前 用兩年也沒遇到問題',
 '後鏡頭一直斷線 拉線要很注意 很麻煩',
 '晚上只有 這種有望遠鏡的才拍的到',
 '邁不錯',
 '現在新的 錄一錄會一直出現記憶卡損壞的語音就停止了 要重裝記憶卡不知道是只有這台',
 '邁後視鏡含夜光後視 六月裝家裡兩台老車 後視都裝在車外 目前都正常 便宜好選擇',
 '我要跟你說 他的導航是廢渣 要用他的導航才能顯示左轉右轉和路名',
 '以前用很久 放 還行',
 '不過用久了 像深灰色款的會慢慢變淡一點',
 '我用了快五年了 持續使用中',
 '我覺得很好用',
 '用過一款 吋包超過十年 很滿意',
 '個人頗推這個牌子好看又耐用',
 '上次買到 折以後約 年還在用',
 '這個沒有 充電呀',
 '我剛好有買這款 因為有三個 插槽才買的 質感還算不錯 平常會外接螢幕使用 會熱但不至於到燙 跟充電都不會影響']

計算字出現的頻率

In [ ]:
X = x_tmp[3]
X

'後鏡頭一直斷線 拉線要很注意 很麻煩'

In [ ]:
X.split()

['後鏡頭一直斷線', '拉線要很注意', '很麻煩']

In [ ]:
''.join(X.split())

'後鏡頭一直斷線拉線要很注意很麻煩'

In [ ]:
egg = ''.join(''.join(x_tmp).split())
egg

'蠻穩定的行車記錄器就是要穩其他其次流媒體不錯晚上又清楚最近有裝畫質不錯之前用兩年也沒遇到問題後鏡頭一直斷線拉線要很注意很麻煩晚上只有這種有望遠鏡的才拍的到邁不錯現在新的錄一錄會一直出現記憶卡損壞的語音就停止了要重裝記憶卡不知道是只有這台邁後視鏡含夜光後視六月裝家裡兩台老車後視都裝在車外目前都正常便宜好選擇我要跟你說他的導航是廢渣要用他的導航才能顯示左轉右轉和路名以前用很久放還行不過用久了像深灰色款的會慢慢變淡一點我用了快五年了持續使用中我覺得很好用用過一款吋包超過十年很滿意個人頗推這個牌子好看又耐用上次買到折以後約年還在用這個沒有充電呀我剛好有買這款因為有三個插槽才買的質感還算不錯平常會外接螢幕使用會熱但不至於到燙跟充電都不會影響'

In [ ]:
count = {}

In [ ]:
for char in egg:
    if char in count.keys():
        count[char] += 1
    else:
        count[char] = 1   

In [ ]:
count

{'一': 5,
 '三': 1,
 '上': 3,
 '不': 8,
 '中': 1,
 '久': 2,
 '之': 1,
 '也': 1,
 '了': 4,
 '五': 1,
 '人': 1,
 '他': 3,
 '以': 2,
 '但': 1,
 '你': 1,
 '使': 2,
 '便': 1,
 '個': 4,
 '停': 1,
 '像': 1,
 '充': 2,
 '光': 1,
 '兩': 2,
 '六': 1,
 '其': 2,
 '出': 1,
 '到': 4,
 '前': 3,
 '剛': 1,
 '包': 1,
 '十': 1,
 '卡': 2,
 '又': 2,
 '只': 2,
 '台': 2,
 '右': 1,
 '吋': 1,
 '名': 1,
 '含': 1,
 '呀': 1,
 '和': 1,
 '問': 1,
 '器': 1,
 '因': 1,
 '在': 3,
 '壞': 1,
 '外': 2,
 '夜': 1,
 '好': 4,
 '媒': 1,
 '子': 1,
 '定': 1,
 '宜': 1,
 '家': 1,
 '導': 2,
 '就': 2,
 '左': 1,
 '常': 2,
 '幕': 1,
 '平': 1,
 '年': 4,
 '廢': 1,
 '影': 1,
 '很': 5,
 '後': 5,
 '得': 1,
 '快': 1,
 '意': 2,
 '感': 1,
 '慢': 2,
 '憶': 2,
 '我': 4,
 '才': 3,
 '折': 1,
 '拉': 1,
 '拍': 1,
 '持': 1,
 '接': 1,
 '推': 1,
 '插': 1,
 '損': 1,
 '擇': 1,
 '放': 1,
 '新': 1,
 '斷': 1,
 '於': 1,
 '是': 3,
 '晚': 2,
 '最': 1,
 '會': 5,
 '月': 1,
 '有': 7,
 '望': 1,
 '楚': 1,
 '槽': 1,
 '次': 2,
 '款': 3,
 '止': 1,
 '正': 1,
 '沒': 2,
 '注': 1,
 '流': 1,
 '淡': 1,
 '深': 1,
 '清': 1,
 '渣': 1,
 '滿': 1,
 '灰': 1,
 '為': 1,
 '煩': 1,
 '熱': 1,
 

給每個字一個編號

In [ ]:
sorted(count, key=count.get,  reverse=True)

['用',
 '的',
 '不',
 '有',
 '要',
 '後',
 '一',
 '很',
 '這',
 '會',
 '錯',
 '裝',
 '年',
 '到',
 '了',
 '好',
 '我',
 '個',
 '車',
 '記',
 '錄',
 '是',
 '他',
 '上',
 '前',
 '鏡',
 '才',
 '在',
 '視',
 '都',
 '還',
 '過',
 '款',
 '買',
 '穩',
 '行',
 '就',
 '其',
 '次',
 '晚',
 '又',
 '質',
 '兩',
 '沒',
 '直',
 '線',
 '意',
 '只',
 '邁',
 '現',
 '憶',
 '卡',
 '台',
 '外',
 '常',
 '跟',
 '導',
 '航',
 '轉',
 '以',
 '久',
 '慢',
 '使',
 '充',
 '電',
 '蠻',
 '定',
 '器',
 '流',
 '媒',
 '體',
 '清',
 '楚',
 '最',
 '近',
 '畫',
 '之',
 '也',
 '遇',
 '問',
 '題',
 '頭',
 '斷',
 '拉',
 '注',
 '麻',
 '煩',
 '種',
 '望',
 '遠',
 '拍',
 '新',
 '出',
 '損',
 '壞',
 '語',
 '音',
 '停',
 '止',
 '重',
 '知',
 '道',
 '含',
 '夜',
 '光',
 '六',
 '月',
 '家',
 '裡',
 '老',
 '目',
 '正',
 '便',
 '宜',
 '選',
 '擇',
 '你',
 '說',
 '廢',
 '渣',
 '能',
 '顯',
 '示',
 '左',
 '右',
 '和',
 '路',
 '名',
 '放',
 '像',
 '深',
 '灰',
 '色',
 '變',
 '淡',
 '點',
 '快',
 '五',
 '持',
 '續',
 '中',
 '覺',
 '得',
 '吋',
 '包',
 '超',
 '十',
 '滿',
 '人',
 '頗',
 '推',
 '牌',
 '子',
 '看',
 '耐',
 '折',
 '約',
 '呀',
 '剛',
 '因',
 '為',
 '三',
 '插',
 '槽',
 '感',
 '算',
 '平'

In [ ]:
egg = sorted(count, key=count.get,  reverse=True)

In [ ]:
for i, char in enumerate(egg, 1):
    print(char, i)

用 1
的 2
不 3
有 4
要 5
後 6
一 7
很 8
這 9
會 10
錯 11
裝 12
年 13
到 14
了 15
好 16
我 17
個 18
車 19
記 20
錄 21
是 22
他 23
上 24
前 25
鏡 26
才 27
在 28
視 29
都 30
還 31
過 32
款 33
買 34
穩 35
行 36
就 37
其 38
次 39
晚 40
又 41
質 42
兩 43
沒 44
直 45
線 46
意 47
只 48
邁 49
現 50
憶 51
卡 52
台 53
外 54
常 55
跟 56
導 57
航 58
轉 59
以 60
久 61
慢 62
使 63
充 64
電 65
蠻 66
定 67
器 68
流 69
媒 70
體 71
清 72
楚 73
最 74
近 75
畫 76
之 77
也 78
遇 79
問 80
題 81
頭 82
斷 83
拉 84
注 85
麻 86
煩 87
種 88
望 89
遠 90
拍 91
新 92
出 93
損 94
壞 95
語 96
音 97
停 98
止 99
重 100
知 101
道 102
含 103
夜 104
光 105
六 106
月 107
家 108
裡 109
老 110
目 111
正 112
便 113
宜 114
選 115
擇 116
你 117
說 118
廢 119
渣 120
能 121
顯 122
示 123
左 124
右 125
和 126
路 127
名 128
放 129
像 130
深 131
灰 132
色 133
變 134
淡 135
點 136
快 137
五 138
持 139
續 140
中 141
覺 142
得 143
吋 144
包 145
超 146
十 147
滿 148
人 149
頗 150
推 151
牌 152
子 153
看 154
耐 155
折 156
約 157
呀 158
剛 159
因 160
為 161
三 162
插 163
槽 164
感 165
算 166
平 167
接 168
螢 169
幕 170
熱 171
但 172
至 173
於 174
燙 175
影 176
響 177


In [ ]:
sorted_char = {char: i for i, char in enumerate(egg, 1)}

In [ ]:
sorted_char['的']

2

In [ ]:
sorted_char[" "] = 0

把一句話用編碼顯示

In [ ]:
X

'後鏡頭一直斷線 拉線要很注意 很麻煩'

In [ ]:
list(map(lambda char:sorted_char[char], X))

[6, 26, 82, 7, 45, 83, 46, 0, 84, 46, 5, 8, 85, 47, 0, 8, 86, 87]

In [ ]:
x = []

for review in x_tmp:
    record = list(map(lambda char:sorted_char[char], review))
    x.append(record)

In [ ]:
y = df["正負評"].values
y

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])